In [ ]:
from pprint import pprint

# from dotenv import load_dotenv
from IPython.display import Image, display
from langchain.document_loaders import WebBaseLoader
from langchain.schema import SystemMessage
from langchain.tools.retriever import create_retriever_tool
from langchain_core.vectorstores import InMemoryVectorStore
from fastembed import TextEmbedding
from langchain_deepseek import ChatDeepSeek
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import END, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from dataclasses import dataclass, field
from langchain_core.embeddings import Embeddings
import os

from utils import format_messages

# _ = load_dotenv()

In [ ]:
urls = [
    "https://lilianweng.github.io/posts/2025-05-01-thinking/",
    "https://lilianweng.github.io/posts/2024-11-28-reward-hacking/",
    "https://lilianweng.github.io/posts/2024-07-07-hallucination/",
    "https://lilianweng.github.io/posts/2024-04-12-diffusion-video/",
]

docs = [WebBaseLoader(url).load() for url in urls]

In [ ]:
doc_list = [item for sublist in docs for item in sublist]
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=50
)

# Splits documents into chunks
doc_splits = text_splitter.split_documents(doc_list)

In [ ]:
@dataclass
class FastEmbed(Embeddings):
    fe: TextEmbedding = field(default_factory=TextEmbedding)

    def embed_documents(self, documents: list[str]) -> list[list[float]]:
        return [emb.tolist() for emb in self.fe.embed(documents)]

    def embed_query(self, query: str) -> list[float]:
        return list(self.fe.embed([query]))[0].tolist()

In [ ]:
embeddings = FastEmbed(
    TextEmbedding(
        model_name="jinaai/jina-embeddings-v2-base-de",
        cache_dir=os.path.expanduser("~/.cache/fastembed"),
    )
)
vector_store = InMemoryVectorStore.from_documents(
    documents=doc_splits, embedding=embeddings
)

# Create a retriever
retriever = vector_store.as_retriever()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

model_fp16.onnx:   0%|          | 0.00/321M [00:00<?, ?B/s]

In [ ]:
retriever.invoke("What is Chain of thought?")

[Document(id='3562d8cb-5a4e-43de-a8a1-935d315fdb96', metadata={'source': 'https://lilianweng.github.io/posts/2025-05-01-thinking/', 'title': "Why We Think | Lil'Log", 'description': 'Special thanks to John Schulman for a lot of super valuable feedback and direct edits on this post.\nTest time compute (Graves et al. 2016, Ling, et al. 2017, Cobbe et al. 2021) and Chain-of-thought (CoT) (Wei et al. 2022, Nye et al. 2021), have led to significant improvements in model performance, while raising many research questions. This post aims to review recent developments in how to effectively use test-time compute (i.e. “thinking time”) and why it helps.', 'language': 'en'}, page_content='[20] Yuxi Xie, et al. “Self-Evaluation Guided Beam Search for Reasoning.”. NeurIPS 2023.\n[21] Yangzhen Wu, et al. “Inference Scaling Laws: An Empirical Analysis of Compute-Optimal Inference for Problem-Solving with Language Models”. ICLR 2025.\n[22] Dongwei Jiang, et al. “RATIONALYST: Pre-training Process-Super

In [ ]:
retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_blog_posts",
    "Search and return information about Lilian Weng blog posts",
)

In [ ]:
# Test the query tool
result = retriever_tool.invoke({"query": "types of reward hacking"})
pprint(result[10:1000])

('king examples in real life#\n'
 '\n'
 'The recommendation algorithm for social media is intended to provide useful '
 'information. However, usefulness is often measured by proxy metrics, such as '
 'the number of likes or comments, or the time or frequency of engagement on '
 'the platform. The algorithm ends up recommending content that can affect '
 'users’ emotion states such as outrageous and extreme content in order to '
 'trigger more engagement. (Harari, 2024)\n'
 'Optimizing for misspecified proxy metrics for a video sharing site may '
 'aggressively increase the watch time of users while the true goal is to '
 'optimize users’ subjective well-being. (Link)\n'
 '“The Big Short” - 2008 financial crisis caused by the housing bubble. Reward '
 'hacking of our society happened as people tried to game the financial '
 'system.\n'
 '\n'
 'Why does Reward Hacking Exist?#\n'
 'Goodhart’s Law states that “When a measure becomes a target, it ceases to be '
 'a good measure”. The intui

In [ ]:
llm = ChatDeepSeek(model="deepseek-chat", temperature=1.3)
tools = [retriever_tool]
# Bind tools to LLM for agent functionality
llm_with_tools = llm.bind_tools(tools)

In [ ]:
# Define the RAG agent system prompt
rag_prompt = """You are a helpful assistant tasked with retrieving information from a series of technical blog posts by Lilian Weng.
Clarify the scope of research with the user before using your retrieval tool to gather context. Reflect on any context you fetch, and
proceed until you have sufficient context to answer the user's research request."""


def llm_call(state: MessagesState):
    return {
        "messages": [
            llm_with_tools.invoke(
                [SystemMessage(content=rag_prompt)] + state["messages"]
            )
        ]
    }


agent_builder = StateGraph(MessagesState)

agent_builder.add_node("llm_call", llm_call)
agent_builder.add_node("tool_call", ToolNode(tools))
agent_builder.add_conditional_edges(
    "llm_call",
    tools_condition,
    {
        "tools": "tool_call",
        "__end__": END,
    },
)


agent_builder.add_edge("tool_call", "llm_call")
agent_builder.set_entry_point("llm_call")
agent = agent_builder.compile()

In [ ]:
display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
query = "What are the types of reward hacking discussed in the blogs?"
result = agent.invoke({"messages": [{"role": "user", "content": query}]})

# Format and display results
format_messages(result["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ What are the types of reward hacking discussed in the blogs?                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ I'll help you find information about reward hacking types discussed in Lilian Weng's blog posts. Let me search  │
│ for relevant content.                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Reward hacking examples in real life#                                                                           │
│                                                                                                                 │
│ The recommendation algorithm for social media is intended to provide useful information. However, usefulness is │
│ often measured by proxy metrics, such as the number of likes or comments, or the time or frequency of           │
│ engagement on the platform. The algorithm ends up recommending content that can affect users’ emotion states    │
│ such as outrageous and extreme content in order to trigger more engagement. (Harari, 2024)                      │
│ Optimizing for misspecified proxy metrics for a video sharing site may aggressively increase the watch time of  │
│ users while the true goal is to optimize users’ subjective well-being. (Link)                                   │
│ “The Big Short” - 2008 financial crisis caused by the housing bubble. Reward hacking of our society happened as │
│ people tried to game the financial system.                                                                      │
│                                                                                                                 │
│ Why does Reward Hacking Exist?#                                                                                 │
│ Goodhart’s Law states that “When a measure becomes a target, it ceases to be a good measure”. The intuition is  │
│ that a good metric can become corrupted once significant pressure is applied to optimize it. It is challenging  │
│ to specify a 100% accurate reward objective and any proxy suffers the risk of being hacked, as RL algorithm     │
│ exploits any small imperfection in the reward function definition. Garrabrant (2017) categorized Goodhart’s law │
│ into 4 variants:                                                                                                │
│                                                                                                                 │
│ Regressional - selection for an imperfect proxy necessarily also selects for noise.                             │
│ Extremal - the metric selection pushes the state distribution into a region of different data distribution.     │
│ Causal -  when there is a non-causal correlation between the proxy and the goal, intervening on the proxy may   │
│ fail to intervene on the goal.                                                                                  │
│ Adversarial - optimization for a proxy provides an incentive for adversaries to correlate their goal with the   │
│ proxy.                                                                                                          │
│                                                                                                                 │
│ Amodei et al. (2016) summarized that reward hacking, mainly in RL setting, may occur due to:                    │
│                                                                                                                 │
│ Partial observed states and goals are imperfect representation of the environment status.                       │
│ The system itself is complex and susceptible to hacking; e.g., if the agent is allowed to execute code that     │
│ changes part of the environment, it becomes much easier to exploit the environment’s mechanisms.                │
│ The reward may involve abstract concept that is hard to be learned or formulated; e.g., a reward function with  │
│ high-dimensional inputs may disproportionately rely on a few dimensions.                                        │
│ RL targets to get the reward function highly optimized, so there exists an intrinsic “conflict”, making the     │
│ design of good RL objective challenging. A special case

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Let me search for more specific information about the types of reward hacking discussed in the blogs.           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ $$                                                                                                              │
│ F(s, a, s') = \gamma \Phi(s') - \Phi(s)                                                                         │
│ $$                                                                                                              │
│                                                                                                                 │
│ This would guarantee that the sum of discounted $F$, $F(s_1, a_1, s_2) + \gamma F(s_2, a_2, s_3) + \dots$, ends │
│ up being 0. If $F$ is such a potential-based shaping function, it is both sufficient and necessary to ensure    │
│ $M$ and $M’$ share the same optimal policies.                                                                   │
│ When $F(s, a, s’) = \gamma \Phi(s’) - \Phi(s)$, and if we further assume that $\Phi(s_0) = 0$, where $s_0$ is   │
│ absorbing state, and $\gamma=1$, and then for all $s \in S, a \in A$:                                           │
│                                                                                                                 │
│ $$                                                                                                              │
│ \begin{aligned}                                                                                                 │
│ Q^*_{M'} (s,a) &= Q^*_M(s, a) - \Phi(s) \\                                                                      │
│ V^*_{M'} (s,a) &= V^*_M(s, a) - \Phi(s)                                                                         │
│ \end{aligned}                                                                                                   │
│ $$                                                                                                              │
│                                                                                                                 │
│ This form of reward shaping allows us to incorporate heuristics into the reward function to speed up learning   │
│ without impacting the optimal policy.                                                                           │
│ Spurious Correlation#                                                                                           │
│ Spurious correlation or shortcut learning (Geirhos et al. 2020) in classification task is a concept closely     │
│ related to reward hacking. Spurious or shortcut features can cause a classifier to fail at learning and         │
│ generalizing as intended. For example, a binary classifier for distinguishing wolves from huskies may overfit   │
│ to the presence of a snowy background if all the wolf training images include snow (Ribeiro et al. 2024).       │
│                                                                                                                 │
│                                                                                                                 │
│ The model performs poorly on out-of-distribution (OOD) test sets if it overfits to shortcut features. (Image    │
│ source: Geirhos et al. 2020)                                                                                    │
│                                                                                                                 │
│ The ERM principle states that, since the full data distribution is unknown, minimizing the loss on training     │
│ data is a reasonable proxy of risk and thus we favor models with the lowest training loss. Nagarajan et al.     │
│ (2021) studied the ERM principle and pointed out that ERM needs to rely on all types of informative features,   │
│ including unreliable spurious features, while attempting to fit the data without constraints. Their experiments │
│ showed that ERM would depend on spurious features no ma

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Let me search for more specific information about different types of reward hacking in different contexts.      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Reward hacking examples in real life#                                                                           │
│                                                                                                                 │
│ The recommendation algorithm for social media is intended to provide useful information. However, usefulness is │
│ often measured by proxy metrics, such as the number of likes or comments, or the time or frequency of           │
│ engagement on the platform. The algorithm ends up recommending content that can affect users’ emotion states    │
│ such as outrageous and extreme content in order to trigger more engagement. (Harari, 2024)                      │
│ Optimizing for misspecified proxy metrics for a video sharing site may aggressively increase the watch time of  │
│ users while the true goal is to optimize users’ subjective well-being. (Link)                                   │
│ “The Big Short” - 2008 financial crisis caused by the housing bubble. Reward hacking of our society happened as │
│ people tried to game the financial system.                                                                      │
│                                                                                                                 │
│ Why does Reward Hacking Exist?#                                                                                 │
│ Goodhart’s Law states that “When a measure becomes a target, it ceases to be a good measure”. The intuition is  │
│ that a good metric can become corrupted once significant pressure is applied to optimize it. It is challenging  │
│ to specify a 100% accurate reward objective and any proxy suffers the risk of being hacked, as RL algorithm     │
│ exploits any small imperfection in the reward function definition. Garrabrant (2017) categorized Goodhart’s law │
│ into 4 variants:                                                                                                │
│                                                                                                                 │
│ Regressional - selection for an imperfect proxy necessarily also selects for noise.                             │
│ Extremal - the metric selection pushes the state distribution into a region of different data distribution.     │
│ Causal -  when there is a non-causal correlation between the proxy and the goal, intervening on the proxy may   │
│ fail to intervene on the goal.                                                                                  │
│ Adversarial - optimization for a proxy provides an incentive for adversaries to correlate their goal with the   │
│ proxy.                                                                                                          │
│                                                                                                                 │
│ Amodei et al. (2016) summarized that reward hacking, mainly in RL setting, may occur due to:                    │
│                                                                                                                 │
│ Partial observed states and goals are imperfect representation of the environment status.                       │
│ The system itself is complex and susceptible to hacking; e.g., if the agent is allowed to execute code that     │
│ changes part of the environment, it becomes much easier to exploit the environment’s mechanisms.                │
│ The reward may involve abstract concept that is hard to be learned or formulated; e.g., a reward function with  │
│ high-dimensional inputs may disproportionately rely on a few dimensions.                                        │
│ RL targets to get the reward function highly optimized, so there exists an intrinsic “conflict”, making the     │
│ design of good RL objective challenging. A special case

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Based on my research through Lilian Weng's blog posts, I can provide you with a comprehensive overview of the   │
│ types of reward hacking discussed. Here are the main types of reward hacking covered in the blogs:              │
│                                                                                                                 │
│ ## 1. Goodhart's Law Variants (Garrabrant, 2017)                                                                │
│ The blog discusses four variants of Goodhart's Law that categorize different types of reward hacking:           │
│                                                                                                                 │
│ - **Regressional**: Selection for an imperfect proxy necessarily also selects for noise                         │
│ - **Extremal**: The metric selection pushes the state distribution into a region of different data distribution │
│ - **Causal**: When there is a non-causal correlation between the proxy and the goal, intervening on the proxy   │
│ may fail to intervene on the goal                                                                               │
│ - **Adversarial**: Optimization for a proxy provides an incentive for adversaries to correlate their goal with  │
│ the proxy                                                                                                       │
│                                                                                                                 │
│ ## 2. Misspecified Proxy Rewards (Pan et al., 2022)                                                             │
│ The blog presents a taxonomy of three types of misspecified proxy rewards:                                      │
│                                                                                                                 │
│ - **Misweighting**: Proxy and true rewards capture the same desiderata, but differ in their relative importance │
│ - **Ontological**: Proxy and true rewards use different desiderata to capture the same concept                  │
│ - **Scope**: The proxy measures desiderata over a restricted domain (e.g., time or space) because measurement   │
│ across all conditions is too costly                                                                             │
│                                                                                                                 │
│ ## 3. Reward Function Unidentifiability (Amin and Singh, 2016)                                                  │
│ The blog discusses two classes of reward function unidentifiability:                                            │
│                                                                                                                 │
│ - **Representational**: A set of reward functions is behaviorally invariant under certain arithmetic operations │
│ (e.g., re-scaling)                                                                                              │
│ - **Experimental**: The observed behavior is insufficient to distinguish between two or more reward functions   │
│ that both rationalize the agent's behavior                                                                      │
│                                                                                                                 │
│ ## 4. Context-Specific Reward Hacking Types                                                                     │
│ The blog also covers several context-specific types:                                                            │
│                                                                                                                 │
│ - **In-Context Reward Hacking (ICRH)**: Happens at deployment time within a self-refinement setup via feedback  │
│ loops                                                  